In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd

# URL of the website
URL = "https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile"

def initialize_driver():
    driver = webdriver.Chrome()
    driver.maximize_window()
    return driver

def load_page(driver, url):
    driver.get(url)
    time.sleep(5)  # Wait for the page to load

# Function to scrape bus routes
def scrape_bus_routes(driver):
    route_elements = driver.find_elements(By.CLASS_NAME, 'route')
    bus_routes_link = [route.get_attribute('href') for route in route_elements]
    bus_routes_name = [route.text.strip() for route in route_elements]
    return bus_routes_link, bus_routes_name

# Function to scrape bus details
def scrape_bus_details(driver, url, route_name):
    try:
        driver.get(url)
        time.sleep(5)  # Allow the page to load
        
        # Click the "View Buses" button if it exists
        try:
            view_buses_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.CLASS_NAME, "button"))
            )
            driver.execute_script("arguments[0].click();", view_buses_button)
            time.sleep(5)  # Wait for buses to load
            
            # Scroll down to load all bus items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(5)  # Wait for the page to load more content

            # Find bus item details
            bus_name_elements = driver.find_elements(By.CLASS_NAME, "travels.lh-24.f-bold.d-color")
            bus_type_elements = driver.find_elements(By.CLASS_NAME, "bus-type.f-12.m-top-16.l-color.evBus")
            departing_time_elements = driver.find_elements(By.CLASS_NAME, "dp-time.f-19.d-color.f-bold")
            duration_elements = driver.find_elements(By.CLASS_NAME, "dur.l-color.lh-24")
            reaching_time_elements = driver.find_elements(By.CLASS_NAME, "bp-time.f-19.d-color.disp-Inline")
            star_rating_elements = driver.find_elements(By.XPATH, "//div[@class='rating-sec lh-24']")
            price_elements = driver.find_elements(By.CLASS_NAME, "fare.d-block")

            # Use XPath to handle both seat availability classes
            seat_availability_elements = driver.find_elements(By.XPATH, "//div[contains(@class, 'seat-left m-top-30') or contains(@class, 'seat-left m-top-16')]")

            bus_details = []
            for i in range(len(bus_name_elements)):
                bus_detail = {
                    "Route_Name": route_name,
                    "Route_Link": url,
                    "Bus_Name": bus_name_elements[i].text,
                    "Bus_Type": bus_type_elements[i].text,
                    "Departing_Time": departing_time_elements[i].text,
                    "Duration": duration_elements[i].text,
                    "Reaching_Time": reaching_time_elements[i].text,
                    "Star_Rating": star_rating_elements[i].text if i < len(star_rating_elements) else '0',
                     "Price": price_elements[i].text,
                    "Seat_Availability": seat_availability_elements[i].text if i < len(seat_availability_elements) else '0'
                }
                bus_details.append(bus_detail)
            return bus_details
        
        except Exception as e:
            print(f"Error occurred while scraping bus details for {url}: {str(e)}")
            return []

    except Exception as e:
        print(f"Error occurred while accessing {url}: {str(e)}")
        return []

# List to hold all bus details
all_bus_details = []

# Function to scrape all pages
def scrape_all_pages():
    for page in range(1, 5):  # There are 4 pages
        try:
            driver = initialize_driver()
            load_page(driver, URL)
            
            if page > 1:
                pagination_tab = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"//div[contains(@class, 'DC_117_pageTabs')][text()='{page}']"))
                )
                driver.execute_script("arguments[0].scrollIntoView();", pagination_tab)
                driver.execute_script("arguments[0].click();", pagination_tab)
                time.sleep(5)  # Wait for the page to load
            
            all_bus_routes_link, all_bus_routes_name = scrape_bus_routes(driver)
            # Iterate over each bus route link and scrape the details
            for link, name in zip(all_bus_routes_link, all_bus_routes_name):
                bus_details = scrape_bus_details(driver, link, name)
                if bus_details:
                    all_bus_details.extend(bus_details)
            

        except Exception as e:
            print(f"Error occurred while accessing page {page}: {str(e)}")

# Scrape routes and details from all pages
scrape_all_pages()


# Convert the list of dictionaries to a DataFrame
df = pd.DataFrame(all_bus_details)

# Save the DataFrame to a CSV file
df.to_csv('kadamba_bus_details.csv', index=False)

# Close the driver
driver.quit()



Error occurred while scraping bus details for https://www.redbus.in/bus-tickets/pune-to-goa: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=127.0.6533.99)
Stacktrace:
	GetHandleVerifier [0x00007FF70CD39642+30946]
	(No symbol) [0x00007FF70CCEE3D9]
	(No symbol) [0x00007FF70CBE6FDA]
	(No symbol) [0x00007FF70CBBCB85]
	(No symbol) [0x00007FF70CC637A7]
	(No symbol) [0x00007FF70CC7A771]
	(No symbol) [0x00007FF70CC5C813]
	(No symbol) [0x00007FF70CC2A6E5]
	(No symbol) [0x00007FF70CC2B021]
	GetHandleVerifier [0x00007FF70CE6F84D+1301229]
	GetHandleVerifier [0x00007FF70CE7BDC7+1351783]
	GetHandleVerifier [0x00007FF70CE72A13+1313971]
	GetHandleVerifier [0x00007FF70CD6DD16+245686]
	(No symbol) [0x00007FF70CCF759F]
	(No symbol) [0x00007FF70CCF3814]
	(No symbol) [0x00007FF70CCF39A2]
	(No symbol) [0x00007FF70CCEA3FF]
	BaseThreadInitThunk [0x00007FF8087C257D+29]
	RtlUserThreadStart [0x00007FF808ACAF28+40]

Error occurred while accessi

NameError: name 'driver' is not defined

In [3]:
df

,Route_Name,Route_Link,Bus_Name,Bus_Type,Departing_Time,Duration,Reaching_Time,Star_Rating,Price,Seat_Availability
0,Goa to Kolhapur(Maharashtra),https://www.redbus.in/bus-tickets/goa-to-kolha...,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,18:00,08h 30m,02:30,3.9,INR 450,13 Seats available
1,Goa to Kolhapur(Maharashtra),https://www.redbus.in/bus-tickets/goa-to-kolha...,Kadamba Transport Corporation Limited (KTCL) -...,Non AC Seater 2+2,19:00,07h 30m,02:30,4.1,INR 420,1 Seat available
2,Goa to Kolhapur(Maharashtra),https://www.redbus.in/bus-tickets/goa-to-kolha...,Atmaram Travels (Dolphin),A/C Sleeper (2+1),15:00,06h 00m,21:00,3.7,INR 650,22 Seats available
3,Goa to Kolhapur(Maharashtra),https://www.redbus.in/bus-tickets/goa-to-kolha...,Neeta tours and travels,AC Sleeper (2+1),15:00,04h 00m,19:00,3.5,INR 600,7 Seats available
4,Goa to Kolhapur(Maharashtra),https://www.redbus.in/bus-tickets/goa-to-kolha...,Atmaram Travels (Dolphin),A/C Sleeper (2+1),15:30,06h 15m,21:45,4.5,INR 599,7 Seats available
...,...,...,...,...,...,...,...,...,...,...
73,Mopa Airport to Margao,https://www.redbus.in/bus-tickets/mopa-airport...,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+3),16:00,00h 25m,16:25,3.7,INR 150,47 Seats available
74,Mopa Airport to Margao,https://www.redbus.in/bus-tickets/mopa-airport...,Kadamba Transport Corporation Limited (KTCL) -...,A/C Seater (2+3),17:30,00h 30m,18:00,3.7,INR 150,47 Seats available
75,Calangute (goa) to Goa,https://www.redbus.in/bus-tickets/calangute-go...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),15:30,00h 31m,16:01,3.7,INR 100,41 Seats available
76,Calangute (goa) to Panaji,https://www.redbus.in/bus-tickets/calangute-go...,Kadamba Transport Corporation Limited (KTCL) -...,AC Seater (2+2),15:30,00h 30m,16:00,3.7,INR 100,41 Seats available
